# QUICK DATA CHECK

* Make sure that you have access to data files (have a look at some examples in `/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-II/PDS_Commissioning/waffles/0_TUTORIAL`)

* If you want to analyze new data, you need to pre-process the decoder first to extract PDS information from the raw `hdf5` file provided by the data acquisition system. Have a look at the `00_HDF5toROOT` scripts for this.

# 1. Import the needed tools

In [ ]:
# Import the needed waffles classes/functions
from waffles.plotting.plot import plot_ChannelWsGrid
import waffles.input_output.raw_root_reader as reader
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid

# Import the needed waffles objects
from waffles.np04_data.ProtoDUNE_HD_APA_maps import APA_map

# 2. Read a WaveformSet object out of a ROOT file

## 2.1. Option 1: Provide a filepath

In [ ]:
# Variable to store the path to the ROOT file
filepath = '/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-II/PDS_Commissioning/waffles/0_TUTORIAL/run26687.root'

# Generate a WaveformSet from the ROOT file
wfset = reader.WaveformSet_from_root_file(  
    filepath,                               # path to the root file 
    'pyroot',                               # library to read (if ROOT, use 'pyroot', if not `uproot`)
    read_full_streaming_data = False,       # if False, read the self-triggered data
    truncate_wfs_to_minimum = False,        # truncate the waveforms to the minimum size
    start_fraction = 0.25,                   # starting fraction for reading
    stop_fraction = 0.75,                    # stoping fraction for reading
    subsample = 2,                          # subsample the data reading (read every other entry)
    verbose = True)                          

## 2.2. Option 2: Provide a folderpath
The reader will look for every root file in such folder, and merge all the data into one WaveformSet object

In [ ]:
# Variable to store the path to the ROOT file
folderpath = '/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-II/PDS_Commissioning/waffles/0_TUTORIAL'

# Generate a WaveformSet from the ROOT files
wfset = reader.WaveformSet_from_root_files( 
    'pyroot',
    folderpath = folderpath,                # path to the folder where the ROOT files are stored
    read_full_streaming_data = True,
    truncate_wfs_to_minimum = False,
    start_fraction = 0.0,
    stop_fraction = 0.1,
    subsample = 2,
    verbose = True)

# 3. Quick inspection of some general information

In [ ]:
# You can have a look on the attributes of the object mywvfset (or any other object) by typing:

help(wfset) # Help on the object and displays the documentation

In [ ]:
# For example, you could display the number of waveforms in the waveform set:
print(f"Number of read waveforms: {len(wfset.waveforms)}")

# Or the number of points per waveform,
print(f"Number of points per waveform: {wfset.points_per_wf}")

# the ADCS array of the i-th waveform in the set,
i = 500
print(f"ADCS array of the {i}-th waveform: {wfset.waveforms[i].adcs}")

# the endpoint and channel from which the i-th waveform in this set was acquired,
print(f"Endpoint: {wfset.waveforms[i].endpoint}, Channel: {wfset.waveforms[i].channel}")

# the number of the run(s) during which the waveforms in this waveform set were acquired,
print(f"Run number(s): {wfset.runs}")

# the endpoints and channels for which there's data in this waveform set,
print(f"Channels with available data per endpoint: {wfset.available_channels}")

In [ ]:
# You can check which APA data is available in the waveform set
# by inspecting the output of WaveformSet.get_set_of_endpoints()

set_of_endpoints = wfset.get_set_of_endpoints()
apas = []

if 104 in set_of_endpoints or 105 in set_of_endpoints or 107 in set_of_endpoints:
    print("There is APA 1 info. in this waveformset.")
    apas.append('apa_1')
if 109 in set_of_endpoints:
    print("There is APA 2 info. in this waveformset.")
    apas.append('apa_2')
if 111 in set_of_endpoints:
    print("There is APA 3 info. in this waveformset.")
    apas.append('apa_3')
if 112 in set_of_endpoints or 113 in set_of_endpoints:
    print("There is APA 4 info. in this waveformset.")
    apas.append('apa_4')

# 4. Plot an APA

In [ ]:
# In wfset, which is an object of the WaveformSet class, all of the waveforms 
# are mixed together. By crafting a ChannelWsGrid object out of the wfset object, 
# the waveforms are ordered according to the physical layout of the given APA.

# Select the APA number you want to plot
apa_no = 3

ordered_wfset = ChannelWsGrid(
    APA_map[apa_no],
    wfset)              # The WaveformSet object to be ordered

In [ ]:
# Once the waveforms are physically order, we can plot them in a grid.

wfs_per_axes = 10

figure = plot_ChannelWsGrid( 
            ordered_wfset,              # The ordered waveform set
            share_x_scale = True,
            share_y_scale = True,
            mode = 'overlay',
            wfs_per_axes = wfs_per_axes,
            verbose = True)

figure.update_layout(   
    width = 1100,
    height= 1200,
    showlegend = True,
    title = {
            'text': f"{wfs_per_axes} waveforms per channel, for APA {apa_no}",
            'font': {
                        'size': 24  # Specify the font size for the title
                    }
        },
    )

figure.show()